This tutorial demonstrates how to add new labels to an ongoing project in real-time. The included example showcases the use of the `<Taxonomy>` tag, which can be applied to the entire document. However, this same principle can also be used for other cases, such as 
- modifying labels for computer vision applications (bounding boxes, polygons, etc.), or segments in text and audio using nested `<Taxonomy>` tags. 
- modify other tags that contain lists of labels, such as `<Choices>` and `<Labels>`.

Here is a list of the necessary items:

In [2]:
# Label Studio instance - replace if in case you use self-hosted app
LABEL_STUDIO_URL = "https://app.heartex.com"

# !! PUT YOUR API KEY HERE - can be retrieved from Account & Settings page
LABEL_STUDIO_API_KEY = '<YOUR-API-KEY>'

# The project ID where you want to add new labels
project_id = 32943

# Taxonomy tag name where labels should be added in each project
tag_name = 'taxonomy'

# JSON payload representing the taxonomy tree to be added
update_taxonomy = {
    'New Top Level class': 1,
    'Eukarya': {
        'Cat': 1
    },
    'New Class': {
        'Object': 1,
        'Nested classes': {
            'And deeper hierarchy': {
                'Object': 1
            },
            'Another Object': 1
        }
    }
}

Here is a function `add_taxonomy_nodes` that supports Label Studio XML tree structure and update the nodes given `update_taxonomy` payload. Note that it adds new non-leaf nodes if they are missing:

In [5]:
from label_studio_sdk import Client
import xml.etree.ElementTree as ET


def add_nodes_recursive(parent, payload, tag_type='Choice'):
    """Helper function to walk recursively over taxonomy tree given current 'parent' node"""
    for key, value in payload.items():
        # Check if the payload value is a dictionary - meaning nested tags
        if isinstance(value, dict):
            # Find the parent tag for nested tags
            nested_parent = parent.find(f".//{tag_type}[@value='{key}']")
            # If parent tag is not found, create it
            if nested_parent is None:
                nested_parent = ET.SubElement(parent, tag_type, {'value': key})
            # Add nested tags recursively
            add_nodes_recursive(nested_parent, value, tag_type)
        else:
            # Add top-level tags directly under the parent tag
            ET.SubElement(parent, tag_type, {'value': key})

def add_new_labels(project_id, tag_name, payload, tag_type='Taxonomy'):
    project = ls.get_project(project_id)
    print(f'Updating project "{project.title}" (ID={project.id})')
    label_config = project.label_config
    root = ET.fromstring(label_config)
    # Locate the desired tag in XML
    tag_to_update = root.find(f'.//{tag_type}[@name="{tag_name}"]')
    if not tag_to_update:
        print(f'No <{tag_type} name="{tag_name}".../> tag found.')
        return
    # Add nodes recursively
    child_tag_type = 'Choice' if tag_type in ('Taxonomy', 'Choices') else 'Label'
    add_nodes_recursive(tag_to_update, payload, child_tag_type)
    new_label_config = ET.tostring(root, encoding='unicode')
    project.set_params(label_config=new_label_config)

Now let's try it in action. Here is the current project configuration - it's taken from the default Templates under [Natural Language Processing > Taxonomy](https://labelstud.io/templates/taxonomy.html)

In [6]:
ls = Client(url=LABEL_STUDIO_URL, api_key=LABEL_STUDIO_API_KEY)

print(ls.get_project(project_id).label_config)

<View>
  <Text name="text" value="$text"/>
  <Taxonomy name="taxonomy" toName="text">
    <Choice value="Archaea" />
    <Choice value="Bacteria" />
    <Choice value="Eukarya">
      <Choice value="Human" />
      <Choice value="Oppossum" />
      <Choice value="Extraterrestial" />
    </Choice>
  </Taxonomy>
</View>


Now let's add new labels from the specified `update_taxonomy` payload:

In [7]:
add_new_labels(project_id, tag_name, update_taxonomy, tag_type='Taxonomy')

Updating project "Taxonomy AutoUpdate" (ID=32943)


In [8]:
print(ls.get_project(project_id).label_config)

<View>
  <Text name="text" value="$text" />
  <Taxonomy name="taxonomy" toName="text">
    <Choice value="Archaea" />
    <Choice value="Bacteria" />
    <Choice value="Eukarya">
      <Choice value="Human" />
      <Choice value="Oppossum" />
      <Choice value="Extraterrestial" />
    <Choice value="Cat" /></Choice>
  <Choice value="New Top Level class" /><Choice value="New Class"><Choice value="Object" /><Choice value="Nested classes"><Choice value="And deeper hierarchy"><Choice value="Object" /></Choice><Choice value="Another Object" /></Choice></Choice></Taxonomy>
</View>


The existing taxonomy tree has been updated with new labels, including nested hierarchical items. Since only new labels were added, this can be done without interrupting ongoing projects.
<div class="alert alert-block alert-warning">
<b>WARNING</b> It's important to be cautious when adding new labels during the annotation process, as it could potentially invalidate previously created data and require you to restart the process. Only proceed with adding new labels if you are certain it won't cause any issues.
</div>

Now you can get all project IDs where you need to add new labels:

In [ ]:
for project in ls.list_projects():
    add_new_labels(project.id, tag_name, update_taxonomy)